In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-ppi45o_h
  Running command git clone --quiet 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-ppi45o_h
  Resolved https://****@github.com/SKTBrain/KoBERT.git to commit e1f2f37055e7460d8427f6912579c0162cb69831
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.5/132.5 KB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 43.2 MB/s eta 0:00:00m eta 0:00:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 55.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 KB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 61.3 MB/s eta 0:00:000m eta 0:00:010:01
  Created wheel for kobert: filename=kobert-0.2.3-py3-none-any.whl size=15663 sha256=70c36378aed2664c88f5a91fd3eb03e2a25351fe6a1145f254e36a6cf08ad3a1
  Stored in directory: /tmp/pip-ephem-wheel-cache-opii

In [22]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model


##GPU 사용 시
device = torch.device("cuda:0")

In [4]:
bertmodel, vocab = get_pytorch_kobert_model()

/home/kyle/course-material/notebooks/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/home/kyle/course-material/notebooks/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [5]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)
# Tokenizer 불러오기
# seq_len을 정해야 한다
trans = nlp.data.BERTSentenceTransform(tok, max_seq_length=64)

using cached model. /home/kyle/course-material/notebooks/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [6]:
!wget https://www.dropbox.com/s/374ftkec978br3d/ratings_train.txt?dl=1
!wget https://www.dropbox.com/s/977gbwh542gdy94/ratings_test.txt?dl=1

--2022-05-14 08:55:07--  https://www.dropbox.com/s/374ftkec978br3d/ratings_train.txt?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.84.18, 2620:100:6034:18::a27d:5412
Connecting to www.dropbox.com (www.dropbox.com)|162.125.84.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/374ftkec978br3d/ratings_train.txt [following]
--2022-05-14 08:55:07--  https://www.dropbox.com/s/dl/374ftkec978br3d/ratings_train.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc5b5579c936a0bb4fd4d802e2fe.dl.dropboxusercontent.com/cd/0/get/BlOV7VDRgoUt_fxkl_JGmh6mq-6ZENo76DQ9pchCHxSVm4Yay6qarsJUzeyjwtjcEbZKaxeXvfXHxEiL0YG4UCBELRlNiHkAj3qF66a8vYO9XyLRYO76kdXsWvg-OKsOAFqJRrszXRTnoT73vSk2V2gCUghw75Pg9VqRrV3W5ZoVyw/file?dl=1# [following]
--2022-05-14 08:55:07--  https://uc5b5579c936a0bb4fd4d802e2fe.dl.dropboxusercontent.com/cd/0/get/BlOV7VDRgoUt_fxkl_JGmh6mq-6ZENo76DQ9pchCHxSVm4Ya

In [12]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, vocab=vocab, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [13]:
# Setting parameters
max_len = 64
batch_size = 64

dataset_test = nlp.data.TSVDataset("ratings_test.txt?dl=1", field_indices=[1,2], num_discard_samples=1)
data_test = BERTDataset(dataset_test, 0, 1, tok, vocab, max_len, True, False)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

In [14]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        self.softmax = nn.Softmax(dim = 1)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.softmax(self.classifier(out))

In [17]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [18]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [19]:
test_acc = 0
for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length= valid_length
    label = label.long().to(device)
    out = model(token_ids, valid_length, segment_ids)
    test_acc += calc_accuracy(out, label)
    if batch_id == 0:
      print(f'output = {out[:3]}')
print("Test acc {}".format(test_acc / (batch_id+1)))

<ipython-input-19-ec03b3c9e46c>:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/782 [00:00<?, ?it/s]

output = tensor([[0.4693, 0.5307],
        [0.3634, 0.6366],
        [0.6216, 0.3784]], device='cuda:0', grad_fn=<SliceBackward0>)
Test acc 0.5010190217391305


## Fine tuning

In [25]:
from transformers import AdamW
from transformers.optimization import  get_cosine_schedule_with_warmup

# Parameters

warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [31]:
dataset_train = nlp.data.TSVDataset("ratings_train.txt?dl=1", field_indices=[1,2], num_discard_samples=1)
data_train = BERTDataset(dataset_train, 0, 1, tok, vocab, max_len, True, False)
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)

In [32]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [33]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [34]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

<ipython-input-34-e6a38b13095b>:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/2344 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 0.7098344564437866 train acc 0.5
epoch 1 batch id 201 loss 0.5460638999938965 train acc 0.5666977611940298
epoch 1 batch id 401 loss 0.48434382677078247 train acc 0.6744856608478803
epoch 1 batch id 601 loss 0.4649675488471985 train acc 0.7239236688851913
epoch 1 batch id 801 loss 0.5512837171554565 train acc 0.7518921660424469
epoch 1 batch id 1001 loss 0.42420637607574463 train acc 0.7658123126873126
epoch 1 batch id 1201 loss 0.49051886796951294 train acc 0.7781015820149875
epoch 1 batch id 1401 loss 0.5262107849121094 train acc 0.7855884189864383
epoch 1 batch id 1601 loss 0.44049566984176636 train acc 0.7925320112429731
epoch 1 batch id 1801 loss 0.44042542576789856 train acc 0.7983932537479178
epoch 1 batch id 2001 loss 0.438401997089386 train acc 0.8039183533233384
epoch 1 batch id 2201 loss 0.477384090423584 train acc 0.8082121762835075
epoch 1 train acc 0.8112867960750854


<ipython-input-34-e6a38b13095b>:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/782 [00:00<?, ?it/s]

...

  0%|          | 0/782 [00:00<?, ?it/s]

epoch 3 test acc 0.8864490089514067


  0%|          | 0/2344 [00:00<?, ?it/s]

epoch 4 batch id 1 loss 0.4850531816482544 train acc 0.828125
epoch 4 batch id 201 loss 0.37437599897384644 train acc 0.9067164179104478
epoch 4 batch id 401 loss 0.3566151261329651 train acc 0.9098347880299252
epoch 4 batch id 601 loss 0.43556728959083557 train acc 0.9124116056572379
epoch 4 batch id 801 loss 0.42909952998161316 train acc 0.9142478152309613
epoch 4 batch id 1001 loss 0.4396789073944092 train acc 0.9150068681318682


KeyboardInterrupt: 